## Preparations

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import numpy as np
from scipy.spatial import distance


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
#device = torch.device("cpu")

## Load & Preprocess Data 

Let's have a look to our dialydialog dataset : https://www.aclweb.org/anthology/I17-1099/

In [2]:
corpus_name = "dailydialog"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding="utf-8") as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "dialogues_text.txt"))

The kitchen stinks . __eou__ I'll throw out the garbage . __eou__

So Dick , how about getting some coffee for tonight ? __eou__ Coffee ? I don ’ t honestly like that kind of stuff . __eou__ Come on , you can at least try a little , besides your cigarette . __eou__ What ’ s wrong with that ? Cigarette is the thing I go crazy for . __eou__ Not for me , Dick . __eou__

Are things still going badly with your houseguest ? __eou__ Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . __eou__ Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . __eou__ You ’ re right . Everything is probably going to come to a head to

## Create formatted data file

We'll create a formatted data file in which each line contains a tab-separated query sentence and a response sentence pair.

The following functions pase the `dailogues_text.txt` data file.
- `loadLines` splits each line of the file into conversations
- `extractSentencePairs` extracts pairs of sentences from conversations

In [3]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName):
    conversations = []
    with open(fileName, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split("__eou__")
            conversations.append(values)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation[i].strip()
            targetLine = conversation[i+1].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

Now we call the above functions to create a new file : `formatted_dialogues_text.txt`

## Load and trim data

Now let's create a vocabulary and load query/response sentence pairs into memory.

First we must create a mapping of each word to a discrete numerical space (the index value).

Voc class keeps the mapping from words to indexes, a reverse mapping of indexes to words, a count of each word and a total word count.
There are 3 central methods :
- `addWord` to add a word to the vocabulary
- `addSentence` to add all words in a sentence
- `trim` for trimming infrequently seen words

In [5]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Before assemble our vocabulary and query/response sentence pairs we must perform some preprocessing.

1. Convert the Unicode strings to ASCII with `unicodeToAscii`.
2. Convert all letters to lowercase and trim all non-letter characters except basic punctuation `normalizeString`
3. Filter out sentences witg length greater than the `MAX_LENGTH` threshold in `filterPairs`


In [6]:
#handle dull_responses now
dull_responses = ["I do not know what you are talking about.", "I do not know.", 
 "You do not know.", "You know what I mean.", "I know what you mean.", 
 "You know what I am saying.", "You do not know anything."]

In [7]:
MAX_LENGTH = 15  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for d in dull_responses:
        voc.addSentence(d)
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

Finally assmble voc and pairs

In [8]:
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 89862 sentence pairs
Trimmed to 43724 sentence pairs
Counting words...
Counted words: 10126

pairs:
['the kitchen stinks .', 'i ll throw out the garbage .']
['so dick how about getting some coffee for tonight ?', 'coffee ? i don t honestly like that kind of stuff .']
['coffee ? i don t honestly like that kind of stuff .', 'come on you can at least try a little besides your cigarette .']
['would you mind waiting a while ?', 'well how long will it be ?']
['i swear i m going to kill you for this .', 'what s wrong ? didn t you think it was fun ? !']
['never ! but thank you for inviting me .', 'come on . you ll feel better after we hit the showers .']
['certainly . how about spaghetti with clams and shrimps .', 'sounds delicious . ok . she ll try that .']
['can you manage chopsticks ?', 'why not ? see .']
['why not ? see .', 'good mastery . how do you like our chinese food ?']
['i m exhausted .', 'okay let s go home .']


### Trimming rarely used words out of vocab

One tactic beneficial to achieve faster convergence during training is trimming rarely used words out of our vocabulary.

1. Trim words used under `MIN_COUNT` threshold using `voc.trim`
2. Filter out pairs with trimmed words

In [9]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 6159 / 10123 = 0.6084
Trimmed from 43724 pairs to 38716, 0.8855 of total


## Prepare Data for Models

BATCH TECHNIQUE

To accomodate sentences of different sizes in the same batch we make our batched input tensor of shape `(max_length, batch_size)` where sentences shorter than the max_length are zeropadded after the `EOS_token`.

- `inputVar` function handles the process of converting sentences to tensor. It returns a tensor of `lengths` for each sequence in the batch for the decoder.
- `outputVar` function performs the same as `inputVar` but returns a binary mask tensor and a maximum target sentence length.
- `batch2TrainData` takes a bunch of pairs and returns the input and target tensors

In [10]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  16,  187,  482,   80,  130],
        [ 270,   15,  259,  139,   16],
        [  62,  983,   15,   13,  423],
        [ 107,  110,  154,  486,   34],
        [  13,  983,   16, 1078,   35],
        [2525,    8,   39,  690,  326],
        [ 251,  270,  124,   15,   15],
        [  62,   15,   30,    2,    2],
        [2083,    2,    2,    0,    0],
        [  15,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([11,  9,  9,  8,  8])
target_variable: tensor([[ 187,  100,   39,   35,   23],
        [ 129,  326,  124,  158,   24],
        [  15,   45,  111,   64,  206],
        [ 107, 2868,   15,  357,   28],
        [   8,  357,    2,   15,  817],
        [ 701,   28,    0,    2,   34],
        [  13,  146,    0,    0,  305],
        [1037, 1278,    0,    0, 1263],
        [  30,  205,    0,    0, 1264],
        [   2,   15,    0,    0,  588],
        [   0,    2,    0,    0,  818],
        [   0,    0,    0,    0,  819],
        

## Seq2Seq

The brain of our chatbot is a sequence to sequence model.

One RNN acts as an _encoder_  which encodes a variable length input sequence to a fixed-length context vector (the final hidden layer of the RNN).
The second RNN is a _decoder_ which takes a s input a word and a context vector and returns a guess for the next word in the sequence.

 ### Encoder

The encoder RNN iterates through tokens and outputs an "output" vector and a "hidden sate" vector. the hidden state vector is passed to the next time step while the output vector is recorder.
The encoder transforms the context it saw at each point in the sequence into a set of points in a high dimensional space. The decoder will use it to generate the outputted word.

We use a bidirectional multi-layered Gated Recurrent Unit.
It gives the advantage of encoding both past and future context !

Computation Graph :
1. Convert word indexed to embeddings
2. Pack padded batch of sequences for RNN module
3. Forward pass through GRU
4. Unpack padding
5. Sum bidirectional GRU outputs
6. Return output and final hidden state

In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

### Decoder

The decoder RNN uses the encoder's context vectors and internal hidden states to generate the next word of the sequence.
It continues generating words until an `EOS_token`.
The problem with a vanilla seq2seq decoder is that if we rely woley on the context vector it will have information loss. (especially with long input sequences).

-> `attention mechanism` allows the decoder to pay attention to certain parts of the input sequence.

In [12]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

Now that the attention submodule is implemented let's dive into the actual decoder model.

Computation Graph:
1. Get embedding of current input word.
2. Forward through unidirectional GRU
3. Calculate attention weights from the current GRU output
4. Multiply attention weights to encoder outputs to get a new context vector
5. Concatenate weighted context vector and GRU using Luong eq
6. Predict next word
7. Return output and final hidden state

In [13]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## Training 

`maskNNLLLoss` calculates the average negative log likelihood of the elements that correspond to a 1 in the mask tensor.

In [14]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Single training iteration (single batch of inputs)

Couple of clever tricks :
- `teacher forcing` at some probability (set by `teacher_forcing_ratio`) current target word is used as the decoder's next input rather than using the decoder's current guess.
- `gradient clipping` commonly used technique for countering the "exploding gradient" problem.

Sequence of operations :
1. Forward pass entire input batch through encoder
2. initialize decoder inputs as SOS_token and hidden state as the encoder's final hidden state
3. Forward input batch sequence through decoder one time step at a time
4. If teacher forcing : set next decoder input as the current target else : set next decoder input as the current decoder ouptput
5. Calculate and accumulate loss
6. Perform backpropagation
7. Clip gradients
8. Update encoder and decoder

In [15]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

Training iterations + save our model to run inferences or continue training !

In [16]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## Gready decoding for generating sentences

Decoding method when training is not using teacher forcing. At each time step we choose the word from decoder_output with the highest softmax value.

In [17]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

## Evaluation of model

In [18]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

## Run model

possible to laod from checkpoint

In [19]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 5000 # 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))
#print(loadFilename)

# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    #checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


## Train !

In [20]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 10000 #4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.7246
Iteration: 2; Percent complete: 0.0%; Average loss: 8.6433
Iteration: 3; Percent complete: 0.0%; Average loss: 8.5292
Iteration: 4; Percent complete: 0.0%; Average loss: 8.3390
Iteration: 5; Percent complete: 0.1%; Average loss: 8.0923
Iteration: 6; Percent complete: 0.1%; Average loss: 7.6373
Iteration: 7; Percent complete: 0.1%; Average loss: 7.1653
Iteration: 8; Percent complete: 0.1%; Average loss: 6.9798
Iteration: 9; Percent complete: 0.1%; Average loss: 6.9105
Iteration: 10; Percent complete: 0.1%; Average loss: 6.7626
Iteration: 11; Percent complete: 0.1%; Average loss: 6.4929
Iteration: 12; Percent complete: 0.1%; Average loss: 6.2568
Iteration: 13; Percent complete: 0.1%; Average loss: 5.9068
Iteration: 14; Percent complete: 0.1%; Average loss: 5.9458
Iteration: 15; Percent complete: 0.1%; Average loss: 5.7623
Iteration: 16; Percent complete: 0.2%

Iteration: 136; Percent complete: 1.4%; Average loss: 4.6853
Iteration: 137; Percent complete: 1.4%; Average loss: 4.5258
Iteration: 138; Percent complete: 1.4%; Average loss: 4.4161
Iteration: 139; Percent complete: 1.4%; Average loss: 4.2554
Iteration: 140; Percent complete: 1.4%; Average loss: 4.3361
Iteration: 141; Percent complete: 1.4%; Average loss: 4.4490
Iteration: 142; Percent complete: 1.4%; Average loss: 4.2833
Iteration: 143; Percent complete: 1.4%; Average loss: 4.5429
Iteration: 144; Percent complete: 1.4%; Average loss: 4.4690
Iteration: 145; Percent complete: 1.5%; Average loss: 4.5086
Iteration: 146; Percent complete: 1.5%; Average loss: 4.4797
Iteration: 147; Percent complete: 1.5%; Average loss: 4.6294
Iteration: 148; Percent complete: 1.5%; Average loss: 4.4982
Iteration: 149; Percent complete: 1.5%; Average loss: 4.5885
Iteration: 150; Percent complete: 1.5%; Average loss: 4.5896
Iteration: 151; Percent complete: 1.5%; Average loss: 4.5417
Iteration: 152; Percent 

Iteration: 272; Percent complete: 2.7%; Average loss: 4.1059
Iteration: 273; Percent complete: 2.7%; Average loss: 4.1955
Iteration: 274; Percent complete: 2.7%; Average loss: 3.9768
Iteration: 275; Percent complete: 2.8%; Average loss: 3.9514
Iteration: 276; Percent complete: 2.8%; Average loss: 4.1492
Iteration: 277; Percent complete: 2.8%; Average loss: 3.9067
Iteration: 278; Percent complete: 2.8%; Average loss: 3.9862
Iteration: 279; Percent complete: 2.8%; Average loss: 4.2991
Iteration: 280; Percent complete: 2.8%; Average loss: 4.1470
Iteration: 281; Percent complete: 2.8%; Average loss: 4.2156
Iteration: 282; Percent complete: 2.8%; Average loss: 4.0781
Iteration: 283; Percent complete: 2.8%; Average loss: 3.9698
Iteration: 284; Percent complete: 2.8%; Average loss: 3.9199
Iteration: 285; Percent complete: 2.9%; Average loss: 4.1438
Iteration: 286; Percent complete: 2.9%; Average loss: 3.8863
Iteration: 287; Percent complete: 2.9%; Average loss: 4.0568
Iteration: 288; Percent 

Iteration: 407; Percent complete: 4.1%; Average loss: 3.9841
Iteration: 408; Percent complete: 4.1%; Average loss: 3.9943
Iteration: 409; Percent complete: 4.1%; Average loss: 3.8018
Iteration: 410; Percent complete: 4.1%; Average loss: 3.8445
Iteration: 411; Percent complete: 4.1%; Average loss: 3.4805
Iteration: 412; Percent complete: 4.1%; Average loss: 3.9433
Iteration: 413; Percent complete: 4.1%; Average loss: 3.8840
Iteration: 414; Percent complete: 4.1%; Average loss: 3.7211
Iteration: 415; Percent complete: 4.2%; Average loss: 3.7776
Iteration: 416; Percent complete: 4.2%; Average loss: 3.7471
Iteration: 417; Percent complete: 4.2%; Average loss: 4.1135
Iteration: 418; Percent complete: 4.2%; Average loss: 3.7261
Iteration: 419; Percent complete: 4.2%; Average loss: 3.8821
Iteration: 420; Percent complete: 4.2%; Average loss: 3.7423
Iteration: 421; Percent complete: 4.2%; Average loss: 3.8398
Iteration: 422; Percent complete: 4.2%; Average loss: 3.6413
Iteration: 423; Percent 

Iteration: 543; Percent complete: 5.4%; Average loss: 3.8523
Iteration: 544; Percent complete: 5.4%; Average loss: 3.6361
Iteration: 545; Percent complete: 5.5%; Average loss: 3.7218
Iteration: 546; Percent complete: 5.5%; Average loss: 3.6235
Iteration: 547; Percent complete: 5.5%; Average loss: 3.6880
Iteration: 548; Percent complete: 5.5%; Average loss: 3.6265
Iteration: 549; Percent complete: 5.5%; Average loss: 3.5070
Iteration: 550; Percent complete: 5.5%; Average loss: 3.6572
Iteration: 551; Percent complete: 5.5%; Average loss: 3.5307
Iteration: 552; Percent complete: 5.5%; Average loss: 3.8645
Iteration: 553; Percent complete: 5.5%; Average loss: 3.4049
Iteration: 554; Percent complete: 5.5%; Average loss: 3.5128
Iteration: 555; Percent complete: 5.5%; Average loss: 3.4835
Iteration: 556; Percent complete: 5.6%; Average loss: 3.8369
Iteration: 557; Percent complete: 5.6%; Average loss: 3.4688
Iteration: 558; Percent complete: 5.6%; Average loss: 3.4391
Iteration: 559; Percent 

...

Iteration: 9388; Percent complete: 93.9%; Average loss: 0.6884
Iteration: 9389; Percent complete: 93.9%; Average loss: 0.5124
Iteration: 9390; Percent complete: 93.9%; Average loss: 0.7605
Iteration: 9391; Percent complete: 93.9%; Average loss: 0.5919
Iteration: 9392; Percent complete: 93.9%; Average loss: 0.6296
Iteration: 9393; Percent complete: 93.9%; Average loss: 0.6628
Iteration: 9394; Percent complete: 93.9%; Average loss: 0.7775
Iteration: 9395; Percent complete: 94.0%; Average loss: 0.6862
Iteration: 9396; Percent complete: 94.0%; Average loss: 0.5596
Iteration: 9397; Percent complete: 94.0%; Average loss: 0.7551
Iteration: 9398; Percent complete: 94.0%; Average loss: 0.5381
Iteration: 9399; Percent complete: 94.0%; Average loss: 0.7202
Iteration: 9400; Percent complete: 94.0%; Average loss: 0.6326
Iteration: 9401; Percent complete: 94.0%; Average loss: 0.6715
Iteration: 9402; Percent complete: 94.0%; Average loss: 0.7525
Iteration: 9403; Percent complete: 94.0%; Average loss:

Iteration: 9521; Percent complete: 95.2%; Average loss: 0.5979
Iteration: 9522; Percent complete: 95.2%; Average loss: 0.7579
Iteration: 9523; Percent complete: 95.2%; Average loss: 0.6192
Iteration: 9524; Percent complete: 95.2%; Average loss: 0.7569
Iteration: 9525; Percent complete: 95.2%; Average loss: 0.7922
Iteration: 9526; Percent complete: 95.3%; Average loss: 0.7588
Iteration: 9527; Percent complete: 95.3%; Average loss: 0.7167
Iteration: 9528; Percent complete: 95.3%; Average loss: 0.6382
Iteration: 9529; Percent complete: 95.3%; Average loss: 0.6552
Iteration: 9530; Percent complete: 95.3%; Average loss: 0.6563
Iteration: 9531; Percent complete: 95.3%; Average loss: 0.4882
Iteration: 9532; Percent complete: 95.3%; Average loss: 0.6982
Iteration: 9533; Percent complete: 95.3%; Average loss: 0.6348
Iteration: 9534; Percent complete: 95.3%; Average loss: 0.6526
Iteration: 9535; Percent complete: 95.3%; Average loss: 0.6923
Iteration: 9536; Percent complete: 95.4%; Average loss:

Iteration: 9653; Percent complete: 96.5%; Average loss: 0.5767
Iteration: 9654; Percent complete: 96.5%; Average loss: 0.6943
Iteration: 9655; Percent complete: 96.5%; Average loss: 0.6334
Iteration: 9656; Percent complete: 96.6%; Average loss: 0.7692
Iteration: 9657; Percent complete: 96.6%; Average loss: 0.7631
Iteration: 9658; Percent complete: 96.6%; Average loss: 0.7099
Iteration: 9659; Percent complete: 96.6%; Average loss: 0.6004
Iteration: 9660; Percent complete: 96.6%; Average loss: 0.7077
Iteration: 9661; Percent complete: 96.6%; Average loss: 0.6872
Iteration: 9662; Percent complete: 96.6%; Average loss: 0.7772
Iteration: 9663; Percent complete: 96.6%; Average loss: 0.7391
Iteration: 9664; Percent complete: 96.6%; Average loss: 0.7583
Iteration: 9665; Percent complete: 96.7%; Average loss: 0.6669
Iteration: 9666; Percent complete: 96.7%; Average loss: 0.5940
Iteration: 9667; Percent complete: 96.7%; Average loss: 0.7053
Iteration: 9668; Percent complete: 96.7%; Average loss:

Iteration: 9786; Percent complete: 97.9%; Average loss: 0.6572
Iteration: 9787; Percent complete: 97.9%; Average loss: 0.6190
Iteration: 9788; Percent complete: 97.9%; Average loss: 0.5885
Iteration: 9789; Percent complete: 97.9%; Average loss: 0.6493
Iteration: 9790; Percent complete: 97.9%; Average loss: 0.7210
Iteration: 9791; Percent complete: 97.9%; Average loss: 0.6241
Iteration: 9792; Percent complete: 97.9%; Average loss: 0.6321
Iteration: 9793; Percent complete: 97.9%; Average loss: 0.6760
Iteration: 9794; Percent complete: 97.9%; Average loss: 0.6778
Iteration: 9795; Percent complete: 98.0%; Average loss: 0.5391
Iteration: 9796; Percent complete: 98.0%; Average loss: 0.6498
Iteration: 9797; Percent complete: 98.0%; Average loss: 0.6139
Iteration: 9798; Percent complete: 98.0%; Average loss: 0.5872
Iteration: 9799; Percent complete: 98.0%; Average loss: 0.7771
Iteration: 9800; Percent complete: 98.0%; Average loss: 0.6426
Iteration: 9801; Percent complete: 98.0%; Average loss:

Iteration: 9917; Percent complete: 99.2%; Average loss: 0.6455
Iteration: 9918; Percent complete: 99.2%; Average loss: 0.5507
Iteration: 9919; Percent complete: 99.2%; Average loss: 0.6372
Iteration: 9920; Percent complete: 99.2%; Average loss: 0.5598
Iteration: 9921; Percent complete: 99.2%; Average loss: 0.6503
Iteration: 9922; Percent complete: 99.2%; Average loss: 0.5785
Iteration: 9923; Percent complete: 99.2%; Average loss: 0.4556
Iteration: 9924; Percent complete: 99.2%; Average loss: 0.7775
Iteration: 9925; Percent complete: 99.2%; Average loss: 0.6308
Iteration: 9926; Percent complete: 99.3%; Average loss: 0.7828
Iteration: 9927; Percent complete: 99.3%; Average loss: 0.6266
Iteration: 9928; Percent complete: 99.3%; Average loss: 0.5637
Iteration: 9929; Percent complete: 99.3%; Average loss: 0.5797
Iteration: 9930; Percent complete: 99.3%; Average loss: 0.6157
Iteration: 9931; Percent complete: 99.3%; Average loss: 0.5516
Iteration: 9932; Percent complete: 99.3%; Average loss:

## RUN AN PLAY

In [21]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting
#evaluateInput(encoder, decoder, searcher, voc)

## Reinforcement learning

### Forward and backward models 

In [22]:
#forward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
#forward_decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
#forward_encoder = forward_encoder.to(device)
#forward_decoder = forward_decoder.to(device)

In [23]:
forward_encoder = encoder
forward_decoder = decoder
forward_encoder = forward_encoder.to(device)
forward_decoder = forward_decoder.to(device)

In [24]:
backward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
backward_decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
backward_encoder = backward_encoder.to(device)
backward_decoder = backward_decoder.to(device)

### Utility functions

In [25]:
def convertResponse(response):
    size1 = len(response)
    size2 = batch_size
    npRes = np.zeros((size1, size2), dtype=np.int64)
    npLengths = np.zeros(size2, dtype=np.int64)
    for i in range(size1):
        prov = response[i].cpu().numpy()
        for j in range(prov.size):
            npLengths[j] = npLengths[j] + 1
            if prov.size > 1:
                npRes[i][j] = prov[j]
            else:
                npRes[i][j] = prov 
    res = torch.from_numpy(npRes)
    lengths = torch.from_numpy(npLengths)
    return res, lengths

In [26]:
def convertTarget(target):
    size1 = len(target)
    size2 = batch_size
    npRes = np.zeros((size1, size2), dtype=np.int64)
    mask = np.zeros((size1, size2), dtype=np.bool_)
    npLengths = np.zeros(size2, dtype=np.int64)
    for i in range(size1):
        prov = target[i].cpu().numpy()
        for j in range(prov.size):
            npLengths[j] = npLengths[j] + 1
            if prov.size > 1:
                npRes[i][j] = prov[j]
            else:
                npRes[i][j] = prov 
                
            if npRes[i][j] > 0:
                mask[i][j] = True
            else:
                mask[i][j] = False
            
    res = torch.from_numpy(npRes)
    lengths = torch.from_numpy(npLengths)
    mask= torch.from_numpy(mask)
    max_target_len = torch.max(lengths) #.detach().numpy()
    return res, mask, max_target_len

In [27]:
def transformTensorToSameShapeAs(tensor, shape):
    size1, size2 = shape
    npNewT = np.zeros((size1, size2), dtype=np.int64)
    npNewMask = np.zeros((size1, size2), dtype=np.bool_)
    tensorSize1, tensorSize2 = tensor.size()
    for i in range(tensorSize1):
        for j in range(tensorSize2):
            npNewT[i][j] = tensor[i][j]
            npNewMask[i][j]= True
    return torch.from_numpy(npNewT), torch.from_numpy(npNewMask)

### Training step for a single iteration 

In [28]:
def RL(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio):
    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")
    
    #Initialize variables
    loss=0
    #print_losses = []
    response=[]
    
    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    
    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    
    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

     # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
     # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            #print_losses.append(mask_loss.item() * nTotal)
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            #print_losses.append(mask_loss.item() * nTotal)
            
            #ni or decoder_output
            response.append(topi)
            
    return loss, max_target_len, response

Let's define the rewards method for each action.

It is composed of 3 different types of rewards :
- `ease of answering` which is basically saying that a turn generated by a machineshould be easy to respond to. 
- `information flow` : We  want  the  agent  to  contribute new information at each turn to keep the dialogue moving and avoid repetitive sequences. 
- `semantic coherence` measures the adequacy of responses to avoid situations in which the generated replies are highly rewarded but are ungrammatical or not coherent. 

In [29]:
def easeOfAnswering(input_variable, lengths, dull_responses, mask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio):
    NS=len(dull_responses)
    r1=0
    for d in dull_responses:
        d, mask, max_target_len = outputVar(d, voc)
        newD, newMask = transformTensorToSameShapeAs(d, input_variable.size())
        #tar, mask, max_target_len = convertTarget(d)
        forward_loss, forward_len, _ = RL(input_variable, lengths, newD, newMask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio)
        # log (1/P(a|s)) = CE  --> log(P(a | s)) = - CE
        if forward_len > 0:
            r1 -= forward_loss / forward_len
    if len(dull_responses) > 0:
        r1 = r1 / NS
    return r1

In [30]:
def informationFlow(responses):
    r2=0
    if(len(responses) > 2):
        #2 representations obtained from the encoder for two consecutive turns pi and pi+1
        h_pi = responses[-3]
        h_pi1 = responses[-1]
        # length of the two vector might not match
        min_length = min(len(h_pi), len(h_pi+1))
        h_pi = h_pi[:min_length]
        h_pi1 = h_pi1[:min_length]
        #cosine similarity 
        #cos_sim = 1 - distance.cosine(h_pi, h_pi1)
        cos_sim = 1 - distance.cdist(h_pi.cpu().numpy(), h_pi1.cpu().numpy(), 'cosine')
        #Handle negative cos_sim
        if np.any(cos_sim <= 0):
            r2 = - cos_sim
        else:
            r2 = - np.log(cos_sim)
        r2 = np.mean(r2)
    return r2

In [31]:
def semanticCoherence(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio):
    #print("IN R3:")
    #print("Input_variable :", input_variable.shape)
    #print("Lengths :", lengths.shape)
    #print("Target_variable :", target_variable.shape)
    #print("Mask :", mask.shape)
    #print("Max_Target_Len :", max_target_len)
    r3 = 0
    forward_loss, forward_len, _ = RL(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
    ep_input, lengths_trans = convertResponse(target_variable)
    #print("ep_input :", ep_input.shape)
    #print("Lengths transformed :", lengths_trans.shape)
    ep_target, mask_trans, max_target_len_trans = convertTarget(input_variable)
    #print("ep_target :", ep_target.shape)
    #print("mask transformed :", mask_trans.shape)
    #print("max_target_len_trans :", max_target_len_trans)
    backward_loss, backward_len, _ = RL(ep_input, lengths_trans, ep_target, mask_trans, max_target_len_trans, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
    if forward_len > 0:
        r3 += forward_loss / forward_len
    if backward_len > 0:
        r3+= backward_loss / backward_len
    return r3

In [32]:
l1=0.25
l2=0.25
l3=0.5

In [33]:
def calculate_rewards(input_var, lengths, target_var, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio):
    #rewards per episode
    ep_rewards = []
    #indice of current episode
    ep_num = 1
    #list of responses
    responses = []
    #input of current episode
    ep_input = input_var
    #target of current episode
    ep_target = target_var
    
    #ep_num bounded -> to redefine (MEDIUM POST)
    while (ep_num <= 10):
        
        print(ep_num)
        #generate current response with the forward model
        _, _, curr_response = RL(ep_input, lengths, ep_target, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Break if :
        # 1 -> dull response
        # 2 -> response is less than MIN_LENGTH
        # 3 -> repetition ie curr_response in responses
        if(len(curr_response) < MIN_COUNT):# or (curr_response in dull_responses) or (curr_response in responses)):
            break
            
            
        #We can add the response to responses list
        #curr_response = torch.LongTensor(curr_response).view(-1, 1)
        #transform curr_response size
        #target = torch.zeros(960, 1)
        #target[:15, :] = curr_response
        #curr_response = target
        #print(curr_response.size())
        #curr_response = torch.reshape(curr_response, (15, 64))
        #print(curr_response.size())
        #curr_response = curr_response.to(device)
        #responses.append(curr_response) 
        
        
        #Ease of answering
        r1 = easeOfAnswering(ep_input, lengths, dull_responses, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Information flow
        r2 = informationFlow(responses)
        
        #Semantic coherence
        r3 = semanticCoherence(ep_input, lengths, target_var, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
        
        #Final reward as a weighted sum of rewards
        r = l1*r1 + l2*r2 + l3*r3
        
        #Add the current reward to the list
        ep_rewards.append(r.detach().cpu().numpy())
        
        #We can add the response to responses list
        curr_response, lengths = convertResponse(curr_response)
        curr_response = curr_response.to(device)
        responses.append(curr_response)
        
        #Next input is the current response
        ep_input = curr_response
        #Next target -> dummy
        ep_target = torch.zeros(MAX_LENGTH,batch_size,dtype=torch.int64)
        #ep_target = torch.LongTensor(torch.LongTensor([0] * MAX_LENGTH)).view(-1, 1)
        ep_target = ep_target.to(device)
        
        #Turn off the teacher forcing  after first iteration -> dummy target
        teacher_forcing_ratio = 0
        ep_num +=1
        
    #Take the mean of the episodic rewards
    return np.mean(ep_rewards) if len(ep_rewards) > 0 else 0 

Dull responses 

In [34]:
dull_responses = ["i do not know what you are talking about.", "i do not know.", 
 "you do not know.", "you know what i mean.", "i know what you mean.", 
 "you know what i am saying.", "you do not know anything."]
#dull_responses = []

### Training RL loop

In [35]:
def trainingRLLoop(model_name, voc, pairs, batch_size, forward_encoder, forward_encoder_optimizer, forward_decoder, forward_decoder_optimizer, backward_encoder, backward_encoder_optimizer, backward_decoder, backward_decoder_optimizer,teacher_forcing_ratio, dull_responses, n_iteration, print_every, save_every, save_dir):
    
    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    
    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    
    
    #Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        print("Iteration", iteration)
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        ##MODIFS HERE
        # Zero gradients the optimizer
        forward_encoder_optimizer.zero_grad()
        forward_decoder_optimizer.zero_grad()
        
        backward_encoder_optimizer.zero_grad()
        backward_decoder_optimizer.zero_grad()
        
        #Forward
        forward_loss, forward_len, _ = RL(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Calculate reward
        reward = calculate_rewards(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
        
        #Update forward seq2seq with loss scaled by reward
        loss = forward_loss * reward
        
        loss.backward()
        forward_encoder_optimizer.step()
        forward_decoder_optimizer.step()

        # Run a training iteration with batch
        print_loss += loss / forward_len
        
        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0
            
        #SAVE CHECKPOINT TO DO
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name)#, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## RUN 

In [36]:
#Configure RL model

model_name='RL_model_seq'
n_iteration = 10000
print_every=100
save_every=500
learning_rate = 0.0001
decoder_learning_ratio = 5.0
teacher_forcing_ratio = 0.5

# Ensure dropout layers are in train mode
forward_encoder.train()
forward_decoder.train()

backward_encoder.train()
backward_decoder.train()

# Initialize optimizers
print('Building optimizers ...')
forward_encoder_optimizer = optim.Adam(forward_encoder.parameters(), lr=learning_rate)
forward_decoder_optimizer = optim.Adam(forward_decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
backward_encoder_optimizer = optim.Adam(backward_encoder.parameters(), lr=learning_rate)
backward_decoder_optimizer = optim.Adam(backward_decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# If you have cuda, configure cuda to call
for state in forward_encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in forward_decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
            
for state in backward_encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in backward_decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
            
# Run training iterations
print("Starting Training!")
trainingRLLoop(model_name, voc, pairs, batch_size, forward_encoder, forward_encoder_optimizer, forward_decoder, forward_decoder_optimizer, backward_encoder, backward_encoder_optimizer, backward_decoder, backward_decoder_optimizer, teacher_forcing_ratio, dull_responses, n_iteration, print_every, save_every, save_dir)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration 1
1
2
3
4
5
6
7
8
9
10
Iteration 2
1
Iteration 3
1
Iteration 4
1
Iteration 5
1
2
3
4
5
6
7
8
9
10
Iteration 6
1
2
3
4
5
6
7
8
9
10
Iteration 7
1
Iteration 8
1
2
3
4
5
6
7
8
9
10
Iteration 9
1
Iteration 10
1
Iteration 11
1
Iteration 12
1
2
3
4
5
6
7
8
9
10
Iteration 13
1
2
3
4
5
6
7
8
9
10
Iteration 14
1
Iteration 15
1
Iteration 16
1
2
3
4
5
6
7
8
9
10
Iteration 17
1
2
3
4
5
6
7
8
9
10
Iteration 18
1
2
3
4
5
6
7
8
9
10
Iteration 19
1
Iteration 20
1
2
3
4
5
6
7
8
9
10
Iteration 21
1
Iteration 22
1
Iteration 23
1
Iteration 24
1
Iteration 25
1
2
3
4
5
6
7
8
9
10
Iteration 26
1
Iteration 27
1
2
3
4
5
6
7
8
9
10
Iteration 28
1
Iteration 29
1
2
3
4
5
6
7
8
9
10
Iteration 30
1
2
3
4
5
6
7
8
9
10
Iteration 31
1
Iteration 32
1
Iteration 33
1
Iteration 34
1
2
3
4
5
6
7
8
9
10
Iteration 35
1
2
3
4
5
6
7
8
9
10
Iteration 36
1
2
3
4
5
6
7
8
9
10
Iteration 37
1
Iteration 38
1
Iteration 39
1
Iteration 40
1
2
3
4
5
6
7
8


1
2
3
4
5
6
7
8
9
10
Iteration 319
1
2
3
4
5
6
7
8
9
10
Iteration 320
1
2
3
4
5
6
7
8
9
10
Iteration 321
1
2
3
4
5
6
7
8
9
10
Iteration 322
1
2
3
4
5
6
7
8
9
10
Iteration 323
1
Iteration 324
1
2
3
4
5
6
7
8
9
10
Iteration 325
1
Iteration 326
1
2
3
4
5
6
7
8
9
10
Iteration 327
1
Iteration 328
1
2
3
4
5
6
7
8
9
10
Iteration 329
1
Iteration 330
1
Iteration 331
1
2
3
4
5
6
7
8
9
10
Iteration 332
1
Iteration 333
1
2
3
4
5
6
7
8
9
10
Iteration 334
1
Iteration 335
1
2
3
4
5
6
7
8
9
10
Iteration 336
1
Iteration 337
1
2
3
4
5
6
7
8
9
10
Iteration 338
1
2
3
4
5
6
7
8
9
10
Iteration 339
1
Iteration 340
1
2
3
4
5
6
7
8
9
10
Iteration 341
1
2
3
4
5
6
7
8
9
10
Iteration 342
1
Iteration 343
1
Iteration 344
1
2
3
4
5
6
7
8
9
10
Iteration 345
1
2
3
4
5
6
7
8
9
10
Iteration 346
1
2
3
4
5
6
7
8
9
10
Iteration 347
1
2
3
4
5
6
7
8
9
10
Iteration 348
1
2
3
4
5
6
7
8
9
10
Iteration 349
1
2
3
4
5
6
7
8
9
10
Iteration 350
1
2
3
4
5
6
7
8
9
10
Iteration 351
1
Iteration 352
1
2
3
4
5
6
7
8
9
10
Iteration 353
1
I

8
9
10
Iteration 630
1
2
3
4
5
6
7
8
9
10
Iteration 631
1
Iteration 632
1
Iteration 633
1
2
3
4
5
6
7
8
9
10
Iteration 634
1
2
3
4
5
6
7
8
9
10
Iteration 635
1
Iteration 636
1
Iteration 637
1
2
3
4
5
6
7
8
9
10
Iteration 638
1
Iteration 639
1
Iteration 640
1
Iteration 641
1
2
3
4
5
6
7
8
9
10
Iteration 642
1
Iteration 643
1
Iteration 644
1
2
3
4
5
6
7
8
9
10
Iteration 645
1
Iteration 646
1
Iteration 647
1
Iteration 648
1
Iteration 649
1
Iteration 650
1
Iteration 651
1
2
3
4
5
6
7
8
9
10
Iteration 652
1
Iteration 653
1
Iteration 654
1
2
3
4
5
6
7
8
9
10
Iteration 655
1
2
3
4
5
6
7
8
9
10
Iteration 656
1
2
3
4
5
6
7
8
9
10
Iteration 657
1
Iteration 658
1
Iteration 659
1
2
3
4
5
6
7
8
9
10
Iteration 660
1
Iteration 661
1
2
3
4
5
6
7
8
9
10
Iteration 662
1
Iteration 663
1
Iteration 664
1
2
3
4
5
6
7
8
9
10
Iteration 665
1
2
3
4
5
6
7
8
9
10
Iteration 666
1
2
3
4
5
6
7
8
9
10
Iteration 667
1
2
3
4
5
6
7
8
9
10
Iteration 668
1
2
3
4
5
6
7
8
9
10
Iteration 669
1
Iteration 670
1
Iteration 671


4
5
6
7
8
9
10
Iteration 953
1
Iteration 954
1
Iteration 955
1
2
3
4
5
6
7
8
9
10
Iteration 956
1
Iteration 957
1
2
3
4
5
6
7
8
9
10
Iteration 958
1
Iteration 959
1
2
3
4
5
6
7
8
9
10
Iteration 960
1
Iteration 961
1
2
3
4
5
6
7
8
9
10
Iteration 962
1
Iteration 963
1
2
3
4
5
6
7
8
9
10
Iteration 964
1
2
3
4
5
6
7
8
9
10
Iteration 965
1
Iteration 966
1
Iteration 967
1
2
3
4
5
6
7
8
9
10
Iteration 968
1
2
3
4
5
6
7
8
9
10
Iteration 969
1
Iteration 970
1
Iteration 971
1
Iteration 972
1
2
3
4
5
6
7
8
9
10
Iteration 973
1
2
3
4
5
6
7
8
9
10
Iteration 974
1
Iteration 975
1
2
3
4
5
6
7
8
9
10
Iteration 976
1
2
3
4
5
6
7
8
9
10
Iteration 977
1
Iteration 978
1
2
3
4
5
6
7
8
9
10
Iteration 979
1
Iteration 980
1
Iteration 981
1
2
3
4
5
6
7
8
9
10
Iteration 982
1
2
3
4
5
6
7
8
9
10
Iteration 983
1
2
3
4
5
6
7
8
9
10
Iteration 984
1
Iteration 985
1
2
3
4
5
6
7
8
9
10
Iteration 986
1
Iteration 987
1
2
3
4
5
6
7
8
9
10
Iteration 988
1
Iteration 989
1
2
3
4
5
6
7
8
9
10
Iteration 990
1
2
3
4
5
6
7
8
9


9
10
Iteration 1255
1
2
3
4
5
6
7
8
9
10
Iteration 1256
1
2
3
4
5
6
7
8
9
10
Iteration 1257
1
2
3
4
5
6
7
8
9
10
Iteration 1258
1
Iteration 1259
1
Iteration 1260
1
2
3
4
5
6
7
8
9
10
Iteration 1261
1
2
3
4
5
6
7
8
9
10
Iteration 1262
1
Iteration 1263
1
Iteration 1264
1
Iteration 1265
1
Iteration 1266
1
Iteration 1267
1
Iteration 1268
1
2
3
4
5
6
7
8
9
10
Iteration 1269
1
Iteration 1270
1
Iteration 1271
1
2
3
4
5
6
7
8
9
10
Iteration 1272
1
2
3
4
5
6
7
8
9
10
Iteration 1273
1
Iteration 1274
1
2
3
4
5
6
7
8
9
10
Iteration 1275
1
Iteration 1276
1
2
3
4
5
6
7
8
9
10
Iteration 1277
1
Iteration 1278
1
Iteration 1279
1
2
3
4
5
6
7
8
9
10
Iteration 1280
1
2
3
4
5
6
7
8
9
10
Iteration 1281
1
Iteration 1282
1
2
3
4
5
6
7
8
9
10
Iteration 1283
1
2
3
4
5
6
7
8
9
10
Iteration 1284
1
2
3
4
5
6
7
8
9
10
Iteration 1285
1
Iteration 1286
1
2
3
4
5
6
7
8
9
10
Iteration 1287
1
Iteration 1288
1
2
3
4
5
6
7
8
9
10
Iteration 1289
1
2
3
4
5
6
7
8
9
10
Iteration 1290
1
2
3
4
5
6
7
8
9
10
Iteration 1291
1
2
3
4

...

10
Iteration: 8500; Percent complete: 85.0%; Average loss: 3.8104
Iteration 8501
1
2
3
4
5
6
7
8
9
10
Iteration 8502
1
2
3
4
5
6
7
8
9
10
Iteration 8503
1
2
3
4
5
6
7
8
9
10
Iteration 8504
1
Iteration 8505
1
2
3
4
5
6
7
8
9
10
Iteration 8506
1
Iteration 8507
1
2
3
4
5
6
7
8
9
10
Iteration 8508
1
Iteration 8509
1
Iteration 8510
1
Iteration 8511
1
2
3
4
5
6
7
8
9
10
Iteration 8512
1
2
3
4
5
6
7
8
9
10
Iteration 8513
1
2
3
4
5
6
7
8
9
10
Iteration 8514
1
2
3
4
5
6
7
8
9
10
Iteration 8515
1
Iteration 8516
1
Iteration 8517
1
Iteration 8518
1
2
3
4
5
6
7
8
9
10
Iteration 8519
1
2
3
4
5
6
7
8
9
10
Iteration 8520
1
Iteration 8521
1
Iteration 8522
1
2
3
4
5
6
7
8
9
10
Iteration 8523
1
Iteration 8524
1
Iteration 8525
1
Iteration 8526
1
Iteration 8527
1
Iteration 8528
1
Iteration 8529
1
Iteration 8530
1
2
3
4
5
6
7
8
9
10
Iteration 8531
1
2
3
4
5
6
7
8
9
10
Iteration 8532
1
Iteration 8533
1
2
3
4
5
6
7
8
9
10
Iteration 8534
1
Iteration 8535
1
2
3
4
5
6
7
8
9
10
Iteration 8536
1
2
3
4
5
6
7
8
9
10

2
3
4
5
6
7
8
9
10
Iteration 8806
1
Iteration 8807
1
Iteration 8808
1
2
3
4
5
6
7
8
9
10
Iteration 8809
1
Iteration 8810
1
Iteration 8811
1
2
3
4
5
6
7
8
9
10
Iteration 8812
1
2
3
4
5
6
7
8
9
10
Iteration 8813
1
Iteration 8814
1
Iteration 8815
1
2
3
4
5
6
7
8
9
10
Iteration 8816
1
Iteration 8817
1
2
3
4
5
6
7
8
9
10
Iteration 8818
1
2
3
4
5
6
7
8
9
10
Iteration 8819
1
Iteration 8820
1
2
3
4
5
6
7
8
9
10
Iteration 8821
1
Iteration 8822
1
Iteration 8823
1
2
3
4
5
6
7
8
9
10
Iteration 8824
1
Iteration 8825
1
2
3
4
5
6
7
8
9
10
Iteration 8826
1
2
3
4
5
6
7
8
9
10
Iteration 8827
1
2
3
4
5
6
7
8
9
10
Iteration 8828
1
Iteration 8829
1
Iteration 8830
1
Iteration 8831
1
2
3
4
5
6
7
8
9
10
Iteration 8832
1
Iteration 8833
1
Iteration 8834
1
Iteration 8835
1
Iteration 8836
1
Iteration 8837
1
2
3
4
5
6
7
8
9
10
Iteration 8838
1
2
3
4
5
6
7
8
9
10
Iteration 8839
1
Iteration 8840
1
Iteration 8841
1
Iteration 8842
1
2
3
4
5
6
7
8
9
10
Iteration 8843
1
Iteration 8844
1
Iteration 8845
1
Iteration 8846
1

1
2
3
4
5
6
7
8
9
10
Iteration 9112
1
Iteration 9113
1
Iteration 9114
1
2
3
4
5
6
7
8
9
10
Iteration 9115
1
2
3
4
5
6
7
8
9
10
Iteration 9116
1
2
3
4
5
6
7
8
9
10
Iteration 9117
1
2
3
4
5
6
7
8
9
10
Iteration 9118
1
2
3
4
5
6
7
8
9
10
Iteration 9119
1
2
3
4
5
6
7
8
9
10
Iteration 9120
1
2
3
4
5
6
7
8
9
10
Iteration 9121
1
2
3
4
5
6
7
8
9
10
Iteration 9122
1
Iteration 9123
1
2
3
4
5
6
7
8
9
10
Iteration 9124
1
Iteration 9125
1
Iteration 9126
1
Iteration 9127
1
Iteration 9128
1
2
3
4
5
6
7
8
9
10
Iteration 9129
1
2
3
4
5
6
7
8
9
10
Iteration 9130
1
2
3
4
5
6
7
8
9
10
Iteration 9131
1
2
3
4
5
6
7
8
9
10
Iteration 9132
1
2
3
4
5
6
7
8
9
10
Iteration 9133
1
Iteration 9134
1
Iteration 9135
1
2
3
4
5
6
7
8
9
10
Iteration 9136
1
Iteration 9137
1
Iteration 9138
1
Iteration 9139
1
Iteration 9140
1
Iteration 9141
1
Iteration 9142
1
Iteration 9143
1
2
3
4
5
6
7
8
9
10
Iteration 9144
1
2
3
4
5
6
7
8
9
10
Iteration 9145
1
2
3
4
5
6
7
8
9
10
Iteration 9146
1
Iteration 9147
1
Iteration 9148
1
Iteratio

2
3
4
5
6
7
8
9
10
Iteration 9411
1
2
3
4
5
6
7
8
9
10
Iteration 9412
1
Iteration 9413
1
2
3
4
5
6
7
8
9
10
Iteration 9414
1
Iteration 9415
1
2
3
4
5
6
7
8
9
10
Iteration 9416
1
2
3
4
5
6
7
8
9
10
Iteration 9417
1
2
3
4
5
6
7
8
9
10
Iteration 9418
1
Iteration 9419
1
2
3
4
5
6
7
8
9
10
Iteration 9420
1
2
3
4
5
6
7
8
9
10
Iteration 9421
1
2
3
4
5
6
7
8
9
10
Iteration 9422
1
Iteration 9423
1
2
3
4
5
6
7
8
9
10
Iteration 9424
1
Iteration 9425
1
Iteration 9426
1
Iteration 9427
1
2
3
4
5
6
7
8
9
10
Iteration 9428
1
Iteration 9429
1
Iteration 9430
1
2
3
4
5
6
7
8
9
10
Iteration 9431
1
2
3
4
5
6
7
8
9
10
Iteration 9432
1
2
3
4
5
6
7
8
9
10
Iteration 9433
1
Iteration 9434
1
2
3
4
5
6
7
8
9
10
Iteration 9435
1
Iteration 9436
1
2
3
4
5
6
7
8
9
10
Iteration 9437
1
2
3
4
5
6
7
8
9
10
Iteration 9438
1
2
3
4
5
6
7
8
9
10
Iteration 9439
1
2
3
4
5
6
7
8
9
10
Iteration 9440
1
2
3
4
5
6
7
8
9
10
Iteration 9441
1
Iteration 9442
1
2
3
4
5
6
7
8
9
10
Iteration 9443
1
2
3
4
5
6
7
8
9
10
Iteration 9444
1
2
3


2
3
4
5
6
7
8
9
10
Iteration 9711
1
2
3
4
5
6
7
8
9
10
Iteration 9712
1
Iteration 9713
1
2
3
4
5
6
7
8
9
10
Iteration 9714
1
2
3
4
5
6
7
8
9
10
Iteration 9715
1
2
3
4
5
6
7
8
9
10
Iteration 9716
1
2
3
4
5
6
7
8
9
10
Iteration 9717
1
Iteration 9718
1
Iteration 9719
1
2
3
4
5
6
7
8
9
10
Iteration 9720
1
Iteration 9721
1
2
3
4
5
6
7
8
9
10
Iteration 9722
1
2
3
4
5
6
7
8
9
10
Iteration 9723
1
2
3
4
5
6
7
8
9
10
Iteration 9724
1
2
3
4
5
6
7
8
9
10
Iteration 9725
1
Iteration 9726
1
Iteration 9727
1
2
3
4
5
6
7
8
9
10
Iteration 9728
1
2
3
4
5
6
7
8
9
10
Iteration 9729
1
2
3
4
5
6
7
8
9
10
Iteration 9730
1
2
3
4
5
6
7
8
9
10
Iteration 9731
1
2
3
4
5
6
7
8
9
10
Iteration 9732
1
2
3
4
5
6
7
8
9
10
Iteration 9733
1
2
3
4
5
6
7
8
9
10
Iteration 9734
1
Iteration 9735
1
2
3
4
5
6
7
8
9
10
Iteration 9736
1
2
3
4
5
6
7
8
9
10
Iteration 9737
1
2
3
4
5
6
7
8
9
10
Iteration 9738
1
2
3
4
5
6
7
8
9
10
Iteration 9739
1
Iteration 9740
1
2
3
4
5
6
7
8
9
10
Iteration 9741
1
Iteration 9742
1
2
3
4
5
6
7
8
9
10


In [37]:
# Set dropout layers to eval mode
forward_encoder.eval()
forward_decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(forward_encoder, forward_decoder)

# Begin chatting
evaluateInput(forward_encoder, forward_decoder, searcher, voc)

> hello
Bot: hi is this the desk ? ? ? ?
> no
Bot: do you have any other ? ? ? ?
> no
Bot: do you have any other ? ? ? ?
> how are you ?
Bot: i m fine . how about you ?
> i am doing well
Bot: where ? are ? ?
> i am at home
Bot: why ? you ?
> i have to work !
Bot: have you to make an ! ? !
> yes i have to
Bot: what a pity ! ? ?
> i agree
Bot: the blue one ? ? it . .
> the sky ?
Bot: yes . we made a computer . . . .
> how ?
Bot: because i want you to say . . .
> haha
Bot: i don t want to . do . . .
> why ?
Bot: because i think it s my finger recently . . .
> how old are you ?
Bot: twenty one . . week . .
> it is quite recent
Error: Encountered unknown word.
> it is quite young
Bot: what does it like ? ? ?
> to ?
Bot: the customer of us will be . by the .
> are you a robot ?
Bot: yes i do . . .
> hi
Bot: hi . here you are . go ? ?
> go
Bot: do you know what s on with ?
> with who ?
Bot: my wife s john . . . . . . .
> what is her name ?
Bot: my name is her . . . .
> hello
Bot: hi is this t

In [38]:
evaluateInput(forward_encoder, forward_decoder, searcher, voc)

> q
